In [33]:
#import tensorflow as tf
#tf.version

<module 'tensorflow._api.v2.version' from 'C:\\Users\\vaio\\AppData\\Roaming\\Python\\Python37\\site-packages\\tensorflow\\_api\\v2\\version\\__init__.py'>

In [37]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

# 1)Construction state

In [38]:
#x1,x2,z are nodes
x1=tf.constant(1)
x2=tf.constant(2)
z=tf.add(x1,x2)

# 2)Evaluation state

In [40]:
sess=tf.Session()

print(sess.run(z))
sess.close()

3


In [41]:
x1 = tf.Variable(1)
x2 = tf.Variable(2)
z = tf.add(x1,x2)

sess =tf.Session()
init=tf.global_variables_initializer()
sess.run(init)
print(sess.run(z))
sess.close()

3


In [11]:
x1=tf.placeholder(tf.float32,2)
x2=tf.placeholder(tf.float32,2)
z = tf.add(x1,x2)

sess=tf.Session()
feed_dict={x1:[1,5],x2:[2,1]}
print(sess.run(z,feed_dict))
sess.close()

[3. 6.]


In [42]:
x1 = tf.placeholder(tf.float32, 1)
w1 = tf.placeholder(tf.float32, 1)
x2 = tf.placeholder(tf.float32, 1)
w2 = tf.placeholder(tf.float32, 1)
z1=tf.multiply(x1,w1)
z2=tf.multiply(x2,w2)
z3=tf.add(z1,z2)

feed_dict={x1:[1],w1:[2],x2:[3],w2:[4]}
sess=tf.Session()
print(sess.run(z3,feed_dict))
sess.close()

[14.]


In [43]:
x1 = tf.constant(2)
x2 = tf.constant(3)
w1 = tf.constant(4)
w2 = tf.constant(5)
z1=tf.multiply(x1,w1)
z2=tf.multiply(x2,w2)

sess=tf.Session()
print(z1.eval(session=sess))
print(z2.eval(session=sess))
print()
sess.close()

8
15



In [44]:
with tf.Session() as sess:
    print(z1.eval())
    print(z2.eval())

8
15


In [45]:
c = tf.constant(5)
x=c+ 1
y=x+ 1
z=x+ 2

# x=tf.add(c,1)
# y=tf.add(x,1)
# z=tf.add()

sess=tf.Session()
print(sess.run([y,z]))

sess.close()

[7, 8]


In [1]:
import numpy as np
def sigmoid(z):
    s=1.0/(1.0+np.exp(-z))
    return s

sigmoid(5)


0.9933071490757153

In [2]:
def tanh(z):
    return np.tanh(z)
tanh(5)

0.9999092042625951

In [3]:
def relu(z):
    return np.maximum(z,0)
relu(5)

5

In [5]:
def lrelu(z,alpha):
    return relu(z)-alpha*relu(-z)
lrelu(-5,0.001)
lrelu(5,0.001)

5.0

# Linear Regression - TF

In [10]:
from sklearn.datasets import load_boston
import numpy as np
boston=load_boston()
#print(boston)
features=np.array(boston.data)
print(features)
labels=np.array(boston.target)
print(labels)
print(features.shape)
print(labels.shape)


[[6.3200e-03 1.8000e+01 2.3100e+00 ... 1.5300e+01 3.9690e+02 4.9800e+00]
 [2.7310e-02 0.0000e+00 7.0700e+00 ... 1.7800e+01 3.9690e+02 9.1400e+00]
 [2.7290e-02 0.0000e+00 7.0700e+00 ... 1.7800e+01 3.9283e+02 4.0300e+00]
 ...
 [6.0760e-02 0.0000e+00 1.1930e+01 ... 2.1000e+01 3.9690e+02 5.6400e+00]
 [1.0959e-01 0.0000e+00 1.1930e+01 ... 2.1000e+01 3.9345e+02 6.4800e+00]
 [4.7410e-02 0.0000e+00 1.1930e+01 ... 2.1000e+01 3.9690e+02 7.8800e+00]]
[24.  21.6 34.7 33.4 36.2 28.7 22.9 27.1 16.5 18.9 15.  18.9 21.7 20.4
 18.2 19.9 23.1 17.5 20.2 18.2 13.6 19.6 15.2 14.5 15.6 13.9 16.6 14.8
 18.4 21.  12.7 14.5 13.2 13.1 13.5 18.9 20.  21.  24.7 30.8 34.9 26.6
 25.3 24.7 21.2 19.3 20.  16.6 14.4 19.4 19.7 20.5 25.  23.4 18.9 35.4
 24.7 31.6 23.3 19.6 18.7 16.  22.2 25.  33.  23.5 19.4 22.  17.4 20.9
 24.2 21.7 22.8 23.4 24.1 21.4 20.  20.8 21.2 20.3 28.  23.9 24.8 22.9
 23.9 26.6 22.5 22.2 23.6 28.7 22.6 22.  22.9 25.  20.6 28.4 21.4 38.7
 43.8 33.2 27.5 26.5 18.6 19.3 20.1 19.5 19.5 20.4 19.8 19.

In [11]:
n_training_samples=features.shape[0]
n_dim=features.shape[1]
print('The dataset has',n_training_samples,'training samples.')
print('The dataset has',n_dim,'features')

The dataset has 506 training samples.
The dataset has 13 features


In [12]:
print(features[0])

[6.320e-03 1.800e+01 2.310e+00 0.000e+00 5.380e-01 6.575e+00 6.520e+01
 4.090e+00 1.000e+00 2.960e+02 1.530e+01 3.969e+02 4.980e+00]


In [13]:
def normalize(dataset):
    mu=np.mean(dataset,axis=0)
    sigma=np.std(dataset,axis=0)
    return(dataset-mu)/sigma
#mu means mean value

In [14]:
mu = np.mean(features, axis = 0)
print(mu)
print(len(mu))

[3.61352356e+00 1.13636364e+01 1.11367787e+01 6.91699605e-02
 5.54695059e-01 6.28463439e+00 6.85749012e+01 3.79504269e+00
 9.54940711e+00 4.08237154e+02 1.84555336e+01 3.56674032e+02
 1.26530632e+01]
13


In [15]:
features_norm=normalize(features)
from sklearn.model_selection import train_test_split
train_x,test_x,train_y,test_y=train_test_split(features_norm,\
                                              labels,test_size=0.33,random_state=42)
print(train_x.shape)
print(test_x.shape)
print(train_y.shape)
print(test_y.shape)

(339, 13)
(167, 13)
(339,)
(167,)


In [16]:
train_x=np.transpose(train_x)
test_x=np.transpose(test_x)
print(train_x.shape)
print(test_x.shape)

(13, 339)
(13, 167)


In [17]:
train_y = np.transpose(train_y)
test_y = np.transpose(test_y)
print(train_y.shape)
print(test_y.shape)

(339,)
(167,)


In [18]:
train_y=train_y.reshape(1,len(train_y))
print(train_y.shape)

(1, 339)


In [19]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
tf.reset_default_graph()

Instructions for updating:
non-resource variables are not supported in the long term


In [23]:
X=tf.placeholder(tf.float32,[n_dim,None])
Y=tf.placeholder(tf.float32,[1,None])
W=tf.Variable(tf.ones([n_dim,1]))
b=tf.Variable(tf.zeros(1))
learning_rate=tf.placeholder(tf.float32,shape=())
print(W)
print(X)
#print(Y)
init=tf.global_variables_initializer()

#construction of nodes using tesnors

y_ = tf.matmul(tf.transpose(W),X)+b
cost=tf.reduce_mean(tf.square(y_-Y))
training_step=tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)
                    

<tf.Variable 'Variable_3:0' shape=(13, 1) dtype=float32_ref>
Tensor("Placeholder_7:0", shape=(13, ?), dtype=float32)


# ---evaluation state-------

In [25]:
def run_linear_model(learning_r,training_epochs,train_obs,train_labels,debug=False):
    sess=tf.Session()
    sess.run(init)
    cost_history=np.empty(shape=[0],dtype=float)
    for epoch in range(training_epochs+1):
        sess.run(training_step,feed_dict={X:train_obs,Y:train_labels,learning_rate:learning_r})
        cost_=sess.run(cost,feed_dict={X:train_obs,Y:train_labels})
        cost_history=np.append(cost_history,cost_)
        if(epoch%10==0)&debug:
            print("Reached epoch", epoch, "cost =",str.format('{0:.6f}',cost_))
        
    return sess,cost_history

sess, cost_history= run_linear_model(learning_r=0.01,
                                                                 
           training_epochs=1000,
                                    train_obs=train_x,
                                    train_labels=train_y,
                                    debug=True)
    
    
    


Reached epoch 0 cost = 624.602173
Reached epoch 10 cost = 376.235352
Reached epoch 20 cost = 252.840805
Reached epoch 30 cost = 175.591003
Reached epoch 40 cost = 125.306023
Reached epoch 50 cost = 92.219353
Reached epoch 60 cost = 70.296379
Reached epoch 70 cost = 55.677929
Reached epoch 80 cost = 45.868675
Reached epoch 90 cost = 39.243568
Reached epoch 100 cost = 34.737511
Reached epoch 110 cost = 31.648687
Reached epoch 120 cost = 29.512304
Reached epoch 130 cost = 28.019245
Reached epoch 140 cost = 26.963037
Reached epoch 150 cost = 26.205223
Reached epoch 160 cost = 25.652580
Reached epoch 170 cost = 25.242109
Reached epoch 180 cost = 24.931047
Reached epoch 190 cost = 24.690237
Reached epoch 200 cost = 24.499680
Reached epoch 210 cost = 24.345591
Reached epoch 220 cost = 24.218390
Reached epoch 230 cost = 24.111368
Reached epoch 240 cost = 24.019793
Reached epoch 250 cost = 23.940266
Reached epoch 260 cost = 23.870333
Reached epoch 270 cost = 23.808189
Reached epoch 280 cost = 2

In [26]:
accuracy=tf.keras.losses.MSE(test_y,y_)
sess.run(accuracy,feed_dict={X:test_x})


array([21.056932], dtype=float32)

In [27]:
sess.close()

# binary classification

In [1]:
#
Cross Entropy H(p,q) =  - 

SyntaxError: invalid syntax (<ipython-input-1-23b5c40bbf67>, line 1)

In [ ]:
from scipy to import loadmat
import numpy as np
import ten
mnist=loadmat("")